Carregar dados e organizar

In [ ]:
import pandas as pd
import numpy as np

url = "https://storage.data.gov.my/transportation/ridership_headline.csv"
df = pd.read_csv(url)

df["date"] = pd.to_datetime(df["date"], errors="raise")
df = df.sort_values("date").set_index("date")

# garante numérico
for c in df.columns:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df.head(), df.shape


Garantir frequência diária e limpar valores inválidos

In [ ]:
# garante um índice diário completo
full_idx = pd.date_range(df.index.min(), df.index.max(), freq="D")
df = df.reindex(full_idx)
df.index.name = "date"

# negativos são inválidos pra contagem
df[df < 0] = np.nan

df.isna().mean().sort_values(ascending=False).head(10)

Preencher buracos curtos

In [ ]:
df_clean = df.copy()

for col in df_clean.columns:
    start = df_clean[col].first_valid_index()
    if start is None:
        continue

    # só preenche a partir do primeiro dado válido (serviço começou a existir/ser reportado)
    s_valid = df_clean.loc[start:, col].interpolate(method="time", limit=7)
    df_clean.loc[start:, col] = s_valid

df_clean.isna().mean().sort_values(ascending=False).head(10)


Criar os alvos: total de ônibus e total de rail

In [ ]:
bus_cols  = ["bus_rkl", "bus_rkn", "bus_rpn"]
rail_cols = [c for c in df_clean.columns if c.startswith("rail_")]

df_model = pd.DataFrame(index=df_clean.index)
df_model["bus_total"]  = df_clean[bus_cols].sum(axis=1, min_count=1)
df_model["rail_total"] = df_clean[rail_cols].sum(axis=1, min_count=1)

# ônibus só existe a partir de 2022-01-01 neste dataset
df_model = df_model.loc["2022-01-01":].copy()

df_model.head(), df_model.isna().sum()


Adicionar features de calendário

In [ ]:
df_feat = df_model.copy()
df_feat["dow"] = df_feat.index.dayofweek
df_feat["is_weekend"] = (df_feat["dow"] >= 5).astype(int)
df_feat["dayofyear"] = df_feat.index.dayofyear

df_feat.head()

Split temporal (80% treino, 20% teste) + normalização sem vazamento

In [ ]:
from sklearn.preprocessing import MinMaxScaler

split = int(len(df_feat) * 0.8)
train = df_feat.iloc[:split]
test  = df_feat.iloc[split:]

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.value)
test_scaled  = scaler.transform(test.values)

feat_cols = train.columns.tolist()
target_cols = ["bus_total", "rail_total"]
target_idx = [feat_cols.index(c) for c in target_cols]

train_scaled.shape, test_scaled.shape


Criar janelas (últimos 14 dias → prever amanhã)

In [ ]:
def make_windows(data, window, target_idx):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window, :])
        y.append(data[i+window, list(target_idx)])
    return np.array(X), np.array(y)

window = 50

all_scaled = np.vstack([train_scaled, test_scaled])
X_all, y_all = make_windows(all_scaled, window=window, target_idx=target_idx)

cut = len(train_scaled) - window
X_train, y_train = X_all[:cut], y_all[:cut]
X_test,  y_test  = X_all[cut:], y_all[cut:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

Treinar uma RNN em PyTorch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# tensores
Xtr = torch.tensor(X_train, dtype=torch.float32)
ytr = torch.tensor(y_train, dtype=torch.float32)
Xte = torch.tensor(X_test, dtype=torch.float32)
yte = torch.tensor(y_test, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=32, shuffle=True)

class RNNModel(nn.Module):
    def __init__(self, n_features, hidden=64, out=2):
        super().__init__()
        self.rnn = nn.RNN(input_size=n_features, hidden_size=hidden, batch_first=True, nonlinearity="tanh")
        # self.rnn = nn.GRU(input_size=n_features, hidden_size=hidden, batch_first=True)
        self.fc = nn.Linear(hidden, out)

    def forward(self, x):
        out, _ = self.rnn(x)      # (batch, time, hidden)
        last = out[:, -1, :]      # último passo
        return self.fc(last)

model = RNNModel(n_features=X_train.shape[2], hidden=64, out=2)
loss_fn = nn.MSELoss()
opt = torch.optim.Adam(model.parameters(), lr=3e-4)

# treino
for epoch in range(50):
    model.train()
    total = 0.0
    for xb, yb in train_loader:
        opt.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        opt.step()
        total += loss.item()
    print(f"epoch {epoch+1:02d} loss {total/len(train_loader):.6f}")


Avaliar no teste (MAE/RMSE) — normalizado e em passageiros reais

* MAE/RMSE normalizado

In [ ]:
model.eval()
with torch.no_grad():
    pred = model(Xte).cpu().numpy()
    true = yte.cpu().numpy()

mae = np.mean(np.abs(pred - true), axis=0)
rmse = np.sqrt(np.mean((pred - true)**2, axis=0))
print("MAE norm [bus, rail]:", mae)
print("RMSE norm [bus, rail]:", rmse)


* Desnormalizar (voltar para passageiros reais)

In [ ]:
def inverse_targets(y_scaled, scaler, n_features, target_idx):
    temp = np.zeros((len(y_scaled), n_features), dtype=np.float32)
    temp[:, target_idx] = y_scaled
    inv = scaler.inverse_transform(temp)
    return inv[:, target_idx]

n_features = len(feat_cols)
pred_orig = inverse_targets(pred, scaler, n_features, target_idx)
true_orig = inverse_targets(true, scaler, n_features, target_idx)

mae_orig = np.mean(np.abs(pred_orig - true_orig), axis=0)
rmse_orig = np.sqrt(np.mean((pred_orig - true_orig)**2, axis=0))

print("MAE original [bus, rail]:", mae_orig)
print("RMSE original [bus, rail]:", rmse_orig)

Baseline (amanhã = hoje) pra comparar

In [ ]:
baseline_pred = X_test[:, -1, :][:, target_idx]  # último dia da janela

mae_b = np.mean(np.abs(baseline_pred - y_test), axis=0)
rmse_b = np.sqrt(np.mean((baseline_pred - y_test)**2, axis=0))

baseline_orig = inverse_targets(baseline_pred, scaler, n_features, target_idx)
ytest_orig = inverse_targets(y_test, scaler, n_features, target_idx)

mae_b_orig = np.mean(np.abs(baseline_orig - ytest_orig), axis=0)
rmse_b_orig = np.sqrt(np.mean((baseline_orig - ytest_orig)**2, axis=0))

print("Baseline MAE original [bus, rail]:", mae_b_orig)
print("Baseline RMSE original [bus, rail]:", rmse_b_orig)

## O que significa “estacionário” (em 30 segundos)

Uma série é estacionária se:

* a média não muda no tempo
* a variância não muda no tempo
* a dependência temporal é estável

📌 ARIMA/SARIMA exigem (quase) estacionariedade.

📌 Passageiros NÃO são estacionários → têm tendência + sazonalidade.

## Teste de estacionariedade (ADF)

In [ ]:
from statsmodels.tsa.stattools import adfuller

y = df_model["bus_total"]   # ou rail_total

# Augmented Dickey-Fuller (ADF)
# H₀: a série não é estacionária
# Queremos p-value < 0.05

adf = adfuller(y.dropna())
print("ADF statistic:", adf[0])
print("p-value:", adf[1])

## Tornar a série estacionária

* Diferenciação (remove tendência)

In [ ]:
y_diff = y.diff().dropna()

adf_diff = adfuller(y_diff)
print("p-value após 1ª diferença:", adf_diff[1])


* Diferenciação sazonal (remove padrão semanal)

In [ ]:
# Como os dados são diários, sazonalidade semanal = 7 dias:

y_seasonal = y.diff(7).dropna()

adf_seasonal = adfuller(y_seasonal)
print("p-value após diferença sazonal:", adf_seasonal[1])

## ACF e PACF (para justificar parâmetros)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, figsize=(12,4))
plot_acf(y_diff, lags=30, ax=ax[0])
plot_pacf(y_diff, lags=30, ax=ax[1])
plt.show()

## Ajustar um modelo SARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(
    y,
    order=(1,1,1),          # ARIMA
    seasonal_order=(1,1,1,7),  # sazonal semanal
    enforce_stationarity=False,
    enforce_invertibility=False
)

res = model.fit(disp=False)
print(res.summary())

## Previsão com SARIMA (dia seguinte)

In [ ]:
forecast = res.get_forecast(steps=1)
pred = forecast.predicted_mean.iloc[0]

print("Previsão SARIMA para amanhã:", pred)


## Avaliar SARIMA (mesma métrica do RNN)

In [ ]:
train_size = int(len(y) * 0.8)
y_train = y.iloc[:train_size]
y_test  = y.iloc[train_size:]

model = SARIMAX(
    y_train,
    order=(1,1,1),
    seasonal_order=(1,1,1,7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

res = model.fit(disp=False)

pred = res.forecast(steps=len(y_test))

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))

mae, rmse


## Gráfico Real vs Previsto – SARIMA

* 1.1 Treino + previsão no período de teste

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
import numpy as np

# série alvo
y = df_model["bus_total"]

# split temporal
split = int(len(y) * 0.8)
y_train = y.iloc[:split]
y_test  = y.iloc[split:]

# SARIMA
model = SARIMAX(
    y_train,
    order=(1,1,1),
    seasonal_order=(1,1,1,7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

res = model.fit(disp=False)

# previsão para o período de teste
y_pred = res.forecast(steps=len(y_test))


* 1.2 Gráfico

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_train.index, y_train, label="Treino", color="gray", alpha=0.6)
plt.plot(y_test.index, y_test, label="Real (Teste)", color="black")
plt.plot(y_test.index, y_pred, label="Previsto (SARIMA)", color="red")

plt.title("SARIMA – Real vs Previsto (Ônibus)")
plt.xlabel("Data")
plt.ylabel("Passageiros")
plt.legend()
plt.tight_layout()
plt.show()

## Gráfico Real vs Previsto – RNN / GRU
* 2.1 Preparar índices corretos do teste

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# garanta que são arrays numpy
true_arr = np.asarray(true_orig)
pred_arr = np.asarray(pred_orig)

# checagens (vai imprimir, mas não trava)
print("df_feat len:", len(df_feat))
print("y_test len:", len(y_test))
print("true_orig shape:", true_arr.shape)
print("pred_orig shape:", pred_arr.shape)

# pega o tamanho que existe em TODOS (evita mismatch)
n = min(len(df_feat.index), len(true_arr), len(pred_arr))
print("Usando n =", n)

# alinha índice e séries pelo mesmo n
idx = df_feat.index[-n:]
true_bus = true_arr[-n:, 0]
pred_bus = pred_arr[-n:, 0]

plt.figure(figsize=(12,5))
plt.plot(idx, true_bus, label="Real (Ônibus)")
plt.plot(idx, pred_bus, label="Previsto (RNN/GRU)")
plt.title("Real vs Previsto (Ônibus) — alinhado")
plt.xlabel("Data")
plt.ylabel("Passageiros")
plt.legend()
plt.tight_layout()
plt.show()


## Gráfico comparativo

In [ ]:
idx = df_feat.index[-n:]
true_rail = true_arr[-n:, 1]
pred_rail = pred_arr[-n:, 1]

plt.figure(figsize=(12,5))
plt.plot(idx, true_rail, label="Real (Rail)")
plt.plot(idx, pred_rail, label="Previsto (RNN/GRU)")
plt.title("Real vs Previsto (Rail) — alinhado")
plt.xlabel("Data")
plt.ylabel("Passageiros")
plt.legend()
plt.tight_layout()
plt.show()